In [ ]:
# Creating a model
embedding_dimensionality = 30
hidden_state_dimensionality = 60

i = tf.keras.layers.Input(shape=(sequence_length,))
x = tf.keras.layers.Embedding(words+1, embedding_dimensionality)(i)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(100)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.models.Model(i, x)

In [4]:
from models.model_utils import load_captioning_model
from tokenizers import KerasTokenizer
import tensorflow as tf
from pipelines import build_pipeline
import os
from tensorboard.plugins import projector

/home/jupyter/MedClip/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
%load_ext tensorboard

In [5]:
model_path = '../models/experiment_2_coco'
model = load_captioning_model(model_path)


2023-02-21 19:41:11.232918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 19:41:11.283972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 19:41:11.286835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 19:41:11.301094: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

1/1 [==============================] - 11s 11s/step - loss: 50.7898 - acc: 0.0000e+00


2023-02-21 19:41:36.915771: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25499376 exceeds 10% of free system memory.
2023-02-21 19:41:37.317032: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25499376 exceeds 10% of free system memory.
2023-02-21 19:41:38.138584: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25499376 exceeds 10% of free system memory.
2023-02-21 19:41:38.335720: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25499376 exceeds 10% of free system memory.
2023-02-21 19:41:39.007146: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25499376 exceeds 10% of free system memory.


In [6]:
tokenizer = KerasTokenizer(model.config['seq_len'])
tokenizer.load_vocabulary(model_path)

In [141]:
model.layers[1].variables[0].numpy().shape

(11802, 512)

In [8]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='./logs/projection-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
    # f.write("vocab\n")
    # f.write("paddingtoken\t\n")
    
    for subwords in tokenizer.vocabulary:
        f.write("{}\n".format('_'+subwords+'_'))
      # Fill in the rest of the labels with "unknown".
    # for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    #     f.write("unknown #{}\n".format(unknown))
    # f.write("flyingcomma\n")

# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[1].variables[0].numpy())
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [155]:
!kill 9072

In [9]:
%tensorboard --logdir './logs/projection-example/'